In [1]:
import requests
import numpy as np
import pandas as pd
import os
import time
import datetime
import re
import warnings
warnings.filterwarnings('ignore')
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
import time
import numpy as np

In [2]:
col_dict = {"corp_id": str, "corp_name": str, "website": str}
corp = pd.read_excel('C:/Users/user/Downloads/corp_website_done.xlsx', dtype=col_dict)

In [3]:
corp['corp_id'] = corp['corp_id'].map(lambda x:("0"*(8-len(x))+x) if len(x)<8 else x)

In [4]:
corp_id_list_total = list(corp["corp_id"])

In [13]:
#selenium settings
options = Options()
options.add_argument("--disable-gpu");
chrome = webdriver.Chrome('./chromedriver',options=options)
#設定等待時間
chrome.implicitly_wait(5)
chrome.set_page_load_timeout(15)

In [16]:
#一次500筆避免觸發反爬蟲(可能是依單一IP總筆數觸發?)
corp_id_list = corp_id_list_total[11000:11500]

In [17]:
corp_id_list[:5]

['81395361', '82796809', '82778350', '81193652', '82797820']

In [18]:
#loop
counter=1
for id in corp_id_list:
    try:
        chrome.get("https://www.twincn.com/item.aspx?no="+str(id))
        try:
            table_tr_list = chrome.find_element("class name", "table-responsive").find_elements("tag name", "tr")
            table_list = []
            for tr in table_tr_list:
                #將每一個tr的資料根據td查詢出來，返回結果為list物件
                table_td_list = tr.find_elements("tag name", "td")
                row_list = []
                for td in table_td_list:    #遍歷每一個td
                    row_list.append(td.text)    #取出表格的資料，並放入行列表裡
                table_list.append(row_list)
            for i in table_list:
                if i[0]=='網址':
                    corp.loc[corp["corp_id"]==id,"website"] = i[1]
                elif i[0]=='公司名稱':
                    corp.loc[corp["corp_id"]==id,"corp_name_web"] = i[1]
                elif i[0]=='商業名稱':
                    corp.loc[corp["corp_id"]==id,"comm_name"] = i[1] 
                elif i[0]=='分公司名稱':
                    corp.loc[corp["corp_id"]==id,"branch_name"] = i[1]                
                elif i[0]=='英文名稱':
                    corp.loc[corp["corp_id"]==id,"corp_name_web_eng"] = i[1]      
                corp.loc[corp["corp_id"]==id,"check"] = "Y"
        except:
            try:
                check_in_homepage = chrome.find_element("id", "search-btn")
                corp.loc[corp["corp_id"]==id,"check"] = "N"
            except:
                corp.loc[corp["corp_id"]==id,"time_out"] = "Y"
                print("觸發反爬蟲,目前資料處理至第{}筆,corp_id為{}".format(corp_id_list_total.index(id),id))
                break
        rand_time = np.random.randint(5)
        time.sleep(rand_time)  #強制等待
        counter = counter+1
        if counter%50==0:
            print("已完成{}筆".format(counter))
            corp.to_excel('C:/Users/user/Downloads/corp_website_done.xlsx',index=False)
    except:
        corp.loc[corp["corp_id"]==id,"time_out"] = "Y"

已完成50筆
已完成100筆
已完成150筆
已完成200筆
已完成250筆
已完成300筆
已完成350筆
已完成400筆
已完成450筆
已完成500筆


In [19]:
corp.to_excel('C:/Users/user/Downloads/corp_website_done.xlsx',index=False)